In [1]:
import pandas as pd
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact

from datetime import timedelta

from src.data_load import load_tables, load_online_instance, load_directorio_hist_df
from src.filtering import flexible_filter
from src.online_algorithms import filter_dfs_for_insertion, get_drivers
from src.metrics import collect_hist_baseline_dfs
from src.experimentation_config import *
from src.config import *
from src.distance_utils import distance

data_path = '../data'

instance = 'instAD1'
distance_method = 'haversine'

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df, labors_static_df, labors_dynamic_df = load_online_instance(data_path, instance, labors_raw_df)
directorio_hist_df = load_directorio_hist_df(data_path, instance)
labors_dynamic_df['latest_arrival_time'] = labors_dynamic_df['schedule_date'] + timedelta(minutes=TIEMPO_GRACIA)

fechas = fechas_dict[instance]

# Upload data

In [3]:
hist_inst = f'{instance[:5]}S{instance[6:]}'
labors_hist_df, moves_hist_df = collect_hist_baseline_dfs(data_path, hist_inst, fechas, distance_method)

In [8]:
import pickle
import os

metrics = ['hybrid']
alphas = [0]

def collect_alpha_results_to_df(data_path: str, instance: str, dist_method: str, metrics: list, alphas: list):
    labors_algo_df = pd.DataFrame()
    moves_algo_df = pd.DataFrame()

    for metric in metrics: 
        for alpha in alphas:
            upload_path = f'{data_path}/resultados/online_operation/{instance}/res_{metric}_{alpha:.1f}_static.pkl'

            if not os.path.exists(upload_path):
                continue
            with open(upload_path, "rb") as f:
                res = pickle.load(f)
                inc_values, duration, results_df, moves_df, metrics_df = res

            if not results_df.empty:
                results_df = results_df.sort_values(["city", "date", "service_id", "labor_id"])
            if not moves_df.empty:
                moves_df = moves_df.sort_values(["city", "date", "service_id", "labor_id"])

            # Normalize datetime columns to Bogotá tz
            datetime_cols = [
                "labor_created_at",
                "labor_start_date",
                "labor_end_date",
                "created_at",
                "schedule_date",
                "actual_start", 
                "actual_end"
                ]


            for df in (results_df, moves_df):
                for col in datetime_cols:
                    if col in df.columns:
                        df[col] = (
                            pd.to_datetime(df[col], errors="coerce", utc=True)
                            .dt.tz_convert("America/Bogota")
                        )
                for col in ['city', 'alfred', 'service_id', 'assigned_driver']:
                    if col in df.columns:
                        df[col] = (
                            df[col]
                            .apply(lambda x: '' if (pd.isna(x) or x=='') else str(int(float(x))))
                        )

            results_df['labor_id'] = (
                results_df['labor_id']
                .apply(lambda x: '' if pd.isna(x) else str(int(float(x))))
            )

            labors_algo_df = pd.concat([labors_algo_df,results_df])
            moves_algo_df = pd.concat([moves_algo_df,moves_df])
    
    return labors_algo_df, moves_algo_df

labors_algo_static_df, moves_algo_static_df = collect_alpha_results_to_df(data_path, instance, 'haversine', metrics, alphas)

# Online assignment

In [ ]:
from src.online_algorithms import evaluate_driver_feasibility
from src.online_algorithms import filter_dynamic_df, filter_dfs_for_new_labor, filter_dfs_for_insertion
from src.online_algorithms import get_drivers, get_best_insertion

labors_algo_dynamic_df = labors_algo_static_df.copy()
moves_algo_dynamic_df = moves_algo_static_df.copy()

unassigned_labors = {}

for city in valid_cities:
    for fecha in fechas:
        unassigned_labors[(city,fecha)] = []

        labors_dynamic_filtered_df = filter_dynamic_df(
            labors_dynamic_df=labors_dynamic_df,
            city=city,
            fecha=fecha
            )
        
        drivers = get_drivers(
            labors_algo_df=labors_algo_static_df,
            city=city,
            fecha=fecha)
        
        for i, new_labor in labors_dynamic_filtered_df.iterrows():
            labors_active_df, moves_active_df = filter_dfs_for_new_labor(
                labors_algo_dynamic_df,
                moves_algo_dynamic_df,
                city=city,
                fecha=fecha,
                created_at=new_labor['created_at']
            )
            
            candidate_insertions = []

            for driver in drivers:          
                labors_driver_df, moves_driver_df = filter_dfs_for_insertion(
                    labors_algo_df=labors_active_df,
                    moves_algo_df=moves_active_df,
                    driver=driver
                    )

                feasible, infeasible_log, insertion_plan = evaluate_driver_feasibility(
                    new_labor=new_labor,
                    driver=driver,
                    moves_driver_df=moves_driver_df,
                    ALFRED_SPEED=ALFRED_SPEED,
                    VEHICLE_TRANSPORT_SPEED=VEHICLE_TRANSPORT_SPEED,
                    TIEMPO_ALISTAR=TIEMPO_ALISTAR,
                    TIEMPO_FINALIZACION=TIEMPO_FINALIZACION, 
                    TIEMPO_GRACIA=TIEMPO_GRACIA,
                    distance_fn=distance
                    )

                if feasible: 
                    candidate_insertions.append((driver, insertion_plan))
            
            if len(candidate_insertions)==0:
                unassigned_labors[(city,fecha)].append(new_labor)
                continue
            
            selected_driver, insertion_point, selection_df = get_best_insertion(
                candidate_insertions, 
                selection_mode="min_total_distance", 
                random_state=None)

            # labors_algo_dynamic_df, moves_algo_dynamic_df = commit_new_labor_insertion()



In [ ]:

unassigned_labors_stat = {key:len(value) for key,value in unassigned_labors.items()}
unassigned_labors_stat

{('149', '2026-01-05'): 0,
 ('149', '2026-01-06'): 0,
 ('149', '2026-01-07'): 0,
 ('149', '2026-01-08'): 0,
 ('149', '2026-01-09'): 0,
 ('149', '2026-01-10'): 0,
 ('149', '2026-01-11'): 0,
 ('1', '2026-01-05'): 0,
 ('1', '2026-01-06'): 0,
 ('1', '2026-01-07'): 0,
 ('1', '2026-01-08'): 0,
 ('1', '2026-01-09'): 0,
 ('1', '2026-01-10'): 0,
 ('1', '2026-01-11'): 0,
 ('126', '2026-01-05'): 1,
 ('126', '2026-01-06'): 0,
 ('126', '2026-01-07'): 0,
 ('126', '2026-01-08'): 0,
 ('126', '2026-01-09'): 0,
 ('126', '2026-01-10'): 0,
 ('126', '2026-01-11'): 0,
 ('150', '2026-01-05'): 3,
 ('150', '2026-01-06'): 2,
 ('150', '2026-01-07'): 3,
 ('150', '2026-01-08'): 1,
 ('150', '2026-01-09'): 1,
 ('150', '2026-01-10'): 1,
 ('150', '2026-01-11'): 0,
 ('844', '2026-01-05'): 0,
 ('844', '2026-01-06'): 0,
 ('844', '2026-01-07'): 3,
 ('844', '2026-01-08'): 0,
 ('844', '2026-01-09'): 1,
 ('844', '2026-01-10'): 1,
 ('844', '2026-01-11'): 0,
 ('830', '2026-01-05'): 0,
 ('830', '2026-01-06'): 0,
 ('830', '2026-

# Previous implementation of helper functions

In [ ]:
# def filter_dynamic_df(labors_dynamic_df, city, fecha):
#     labors_dynamic_filtered_df = flexible_filter(
#         labors_dynamic_df,
#         city=city,
#         schedule_date=fecha
#         ).sort_values(['created_at', 'schedule_date', 'labor_start_date']).reset_index(drop=True)

#     return labors_dynamic_filtered_df


# def get_drivers(labors_algo_df, city, fecha):
#     labors_algo_filtered_df = flexible_filter(
#         labors_algo_df,
#         city=city,
#         schedule_date=fecha)    
#     drivers = labors_algo_filtered_df['assigned_driver'].unique().tolist()

#     return drivers


# def filter_dfs_for_new_labor(labors_algo_df, moves_algo_df, city, fecha, created_at):
#     """
#     Filters labors and moves dataframes based on city, date, and created_at timestamp.
#     """
    
#     def _filter_and_sort(df):
#         # 1. Apply dynamic filters
#         df = flexible_filter(df, city=city, schedule_date=fecha)
#         # 2. Filter and sort
#         df = (
#             df[df["actual_end"] > created_at]
#             .sort_values(["schedule_date", "actual_start"])
#             .reset_index(drop=True)
#         )
#         return df

#     # Apply to both
#     labors_algo_filtered_df = _filter_and_sort(labors_algo_df)
#     moves_algo_filtered_df = _filter_and_sort(moves_algo_df)

#     return labors_algo_filtered_df, moves_algo_filtered_df

    
# def filter_dfs_for_insertion(labors_algo_df, moves_algo_df, driver):
#     labors_algo_filtered_df = flexible_filter(
#         labors_algo_df,
#         assigned_driver=driver
#         ).sort_values(['schedule_date', 'actual_start']).reset_index(drop=True)

#     moves_algo_filtered_df = flexible_filter(
#         moves_algo_df,
#         assigned_driver=driver
#         ).sort_values(['schedule_date', 'actual_start']).reset_index(drop=True)
    
#     return labors_algo_filtered_df, moves_algo_filtered_df


# # ============================================================
# # --- Helper Functions --------------------------------------
# # ============================================================

# def get_driver_context(moves_driver_df, idx):
#     """Return the current and next labor context for driver."""
#     curr_end_time = moves_driver_df.loc[idx, 'actual_end']
#     curr_end_pos = moves_driver_df.loc[idx, 'end_point']
#     next_start_time = moves_driver_df.loc[idx + 3, 'schedule_date']
#     next_start_pos = moves_driver_df.loc[idx + 3, 'start_point']
#     return curr_end_time, curr_end_pos, next_start_time, next_start_pos


# def compute_arrival_time(current_end_time, current_end_pos, target_pos, speed, distance_fn):
#     """Compute when driver would arrive at the target position."""
#     dist, _ = distance_fn(current_end_pos, target_pos, method='haversine')
#     travel_time = dist / speed * 60
#     return current_end_time + timedelta(minutes=travel_time), dist, travel_time


# def adjust_for_early_arrival(would_arrive_at, scheduled_date, early_buffer=30):
#     """
#     Adjusts arrival time if driver would arrive too early.
#     Ensures driver waits to arrive no earlier than (schedule_date - early_buffer).
#     """
#     earliest_allowed = scheduled_date - timedelta(minutes=early_buffer)
#     return max(would_arrive_at, earliest_allowed)


# def compute_service_end_time(arrival_time, start_pos, end_pos, 
#                              vehicle_speed, prep_time, finish_time, distance_fn):
#     """Compute finish time and position of performing the new service."""
#     dist, _ = distance_fn(start_pos, end_pos, method='haversine')
#     travel_time = dist / vehicle_speed * 60
#     total_duration = prep_time + travel_time + finish_time
#     finish_time = arrival_time + timedelta(minutes=total_duration)
#     return finish_time, end_pos, total_duration, dist


# def can_reach_next_labor(new_finish_time, new_finish_pos, next_start_time, next_start_pos, 
#                          driver_speed, grace_time, distance_fn):
#     """Check if driver can arrive to next labor in time after finishing new service."""
#     dist, _ = distance_fn(new_finish_pos, next_start_pos, method='haversine')
#     travel_time = dist / driver_speed * 60
#     would_arrive_next = new_finish_time + timedelta(minutes=travel_time)
#     feasible = would_arrive_next <= next_start_time + timedelta(minutes=grace_time)
#     return feasible, would_arrive_next, dist, travel_time


# # ============================================================
# # --- Main Evaluation Function -------------------------------
# # ============================================================

# def evaluate_driver_feasibility(
#     new_labor,
#     moves_driver_df,
#     ALFRED_SPEED,
#     VEHICLE_TRANSPORT_SPEED,
#     TIEMPO_ALISTAR,
#     TIEMPO_FINALIZACION,
#     TIEMPO_GRACIA,
#     distance_fn,
#     EARLY_BUFFER=30
# ):
#     """
#     Evaluate if a driver can feasibly insert a new labor into their current schedule.

#     Returns
#     -------
#     feasible : bool
#         Whether insertion is feasible for this driver.
#     infeasible_log : str
#         Explanation if infeasible.
#     prev_labor_id : str or None
#         labor_id of the labor before the potential insertion point.
#     next_labor_id : str or None
#         labor_id of the labor after the potential insertion point.
#     dist_to_new_service : float or None
#         Distance (km) from previous endpoint to new labor start.
#     dist_to_next_labor : float or None
#         Distance (km) from new labor end to next scheduled labor start.
#     """

#     infeasible_log = ''
#     feasible = False
#     prev_labor_id = None
#     next_labor_id = None
#     dist_to_new_service = None
#     dist_to_next_labor = None

#     # --- Early exit: driver has no labors
#     if moves_driver_df.empty:
#         return False, "Driver has no scheduled labors.", None, None, None, None

#     labor_iter = 0
#     n_rows = len(moves_driver_df)

#     # Iterate across each real labor (skipping _free and _move)
#     while labor_iter < n_rows - 3:

#         # 1. Get current + next labor context
#         curr_end_time, curr_end_pos, next_start_time, next_start_pos = \
#             get_driver_context(moves_driver_df, labor_iter)

#         curr_labor_id = moves_driver_df.loc[labor_iter, "labor_id"]
#         next_labor_id_candidate = moves_driver_df.loc[labor_iter + 3, "labor_id"]

#         # 2. Skip if next labor starts before new labor’s schedule
#         if next_start_time <= new_labor['schedule_date']:
#             labor_iter += 3
#             continue

#         # 3. Compute arrival time to new service
#         would_arrive_at, dist_to_new_service, travel_time_to_new = compute_arrival_time(
#             curr_end_time, curr_end_pos, new_labor['start_address_point'],
#             ALFRED_SPEED, distance_fn
#         )

#         # 4. Check if can arrive within allowed window
#         if would_arrive_at > new_labor['latest_arrival_time']:
#             infeasible_log = "Driver would not arrive on time to the new labor."
#             break

#         # 5. Adjust if arriving too early (driver waits)
#         real_arrival_time = adjust_for_early_arrival(
#             would_arrive_at, new_labor['schedule_date'], EARLY_BUFFER
#         )

#         # 6. Compute when driver would finish the new labor
#         finish_new_labor_time, finish_new_labor_pos, _, dist_service = \
#             compute_service_end_time(
#                 real_arrival_time,
#                 new_labor['start_address_point'],
#                 new_labor['end_address_point'],
#                 VEHICLE_TRANSPORT_SPEED,
#                 TIEMPO_ALISTAR,
#                 TIEMPO_FINALIZACION,
#                 distance_fn
#             )

#         # 7. Check feasibility to reach next scheduled labor
#         feasible_next, would_arrive_next, dist_to_next_labor, travel_time_to_next = \
#             can_reach_next_labor(
#                 finish_new_labor_time, finish_new_labor_pos,
#                 next_start_time, next_start_pos,
#                 ALFRED_SPEED, TIEMPO_GRACIA, distance_fn
#             )

#         if not feasible_next:
#             infeasible_log = (
#                 "Driver would not make it to the next scheduled labor in time "
#                 "if new labor is inserted."
#             )
#             break

#         # Feasible insertion point found
#         feasible = True
#         prev_labor_id = curr_labor_id
#         next_labor_id = next_labor_id_candidate
#         break

#     # 8. Case: new labor occurs after all existing ones → append at end
#     if not feasible and infeasible_log == '' and labor_iter >= n_rows - 3:
#         feasible = True
#         prev_labor_id = moves_driver_df.loc[n_rows - 1, "labor_id"]
#         next_labor_id = None
#         infeasible_log = ''
#         dist_to_new_service = None
#         dist_to_next_labor = None

#     return feasible, infeasible_log, prev_labor_id, next_labor_id, dist_to_new_service, dist_to_next_labor


# def get_best_insertion(candidate_insertions, selection_mode="min_total_distance", random_state=None):
#     """
#     Selects the best driver among feasible insertions based on a chosen criterion.

#     Parameters
#     ----------
#     candidate_insertions : list of tuples
#         Each tuple should be:
#         (driver, insertion_point, dist_to_new_labor, dist_to_next_labor)
#     selection_mode : str, optional
#         Selection criterion:
#         - "random": choose a random driver
#         - "min_total_distance": minimize (dist_to_new_labor + dist_to_next_labor)
#         - "min_dist_to_new_labor": minimize distance to the new labor
#     random_state : int, optional
#         For reproducibility when using random selection.

#     Returns
#     -------
#     selected_driver : str
#         The chosen driver ID.
#     insertion_point : int
#         Where to insert the new labor in the driver's schedule.
#     selection_df : pd.DataFrame
#         Table summarizing all candidate metrics (for analysis/debugging).
#     """

#     if len(candidate_insertions) == 0:
#         return None, None, pd.DataFrame()

#     # Convert to DataFrame for easier computation
#     selection_df = pd.DataFrame(candidate_insertions, columns=[
#         "driver", "prev_labor_id", 'next_labor_id', "dist_to_new_labor", "dist_to_next_labor"
#     ])

#     # Replace None/NaN manually using np.where (bypasses .fillna())
#     selection_df["dist_to_new_labor"] = np.where(
#         selection_df["dist_to_new_labor"].isna(),
#         0,
#         selection_df["dist_to_new_labor"]
#     ).astype(float)

#     selection_df["dist_to_next_labor"] = np.where(
#         selection_df["dist_to_next_labor"].isna(),
#         0,
#         selection_df["dist_to_next_labor"]
#     ).astype(float)

#     selection_df["total_distance"] = (
#         selection_df["dist_to_new_labor"] + selection_df["dist_to_next_labor"]
#     )

#     # --- Selection logic ---
#     if selection_mode == "random":
#         if random_state is not None:
#             random.seed(random_state)
#         chosen_row = selection_df.sample(1, random_state=random_state).iloc[0]

#     elif selection_mode == "min_dist_to_new_labor":
#         chosen_row = selection_df.loc[selection_df["dist_to_new_labor"].idxmin()]

#     elif selection_mode == "min_total_distance":
#         chosen_row = selection_df.loc[selection_df["total_distance"].idxmin()]

#     else:
#         raise ValueError(f"Unknown selection_mode '{selection_mode}'")

#     selected_driver = chosen_row["driver"]
#     insertion_point = (chosen_row["prev_labor_id"], chosen_row['next_labor_id'])

#     return selected_driver, insertion_point, selection_df




# Control example

In [33]:
moves_algo_static_df.columns

Index(['service_id', 'labor_id', 'labor_context_id', 'labor_name',
       'labor_category', 'assigned_driver', 'schedule_date', 'actual_start',
       'actual_end', 'start_point', 'end_point', 'distance_km', 'duration_min',
       'city', 'date'],
      dtype='object')

In [2]:
city = '149'
date = '2026-01-08'

In [4]:
labors_dynamic_filtered_df = flexible_filter(
    labors_dynamic_df,
    city=city,
    schedule_date=date)

### Nuevo servicio llega a las 7:52 a.m. para las 10:30 a.m. En este caso es una sola labor de transporte.

In [19]:
new_service = labors_dynamic_filtered_df.iloc[2,:]

In [20]:
print(f'service_id: \t{new_service["service_id"]}')
print(f'labor_id: \t{new_service["labor_id"]}')
print(f'created_at: \t{new_service["created_at"].time()}')
print(f'schedule_date: \t{new_service["schedule_date"].time()}')
# print(f': {new_service[""]}')
# print(f': {new_service[""]}')
# print(f': {new_service[""]}')


service_id: 	254696
labor_id: 	350832
created_at: 	08:11:01.202000
schedule_date: 	12:00:00


### Tomar un conductor y revisar sus labores

In [ ]:
drivers = get_drivers(
            labors_algo_df=labors_algo_static_df,
            city=city,
            fecha=date)

driver = drivers[11]
print(f'Driver: {driver}')

Driver: 11988


In [31]:
df = flexible_filter(moves_algo_static_df,
                city='149',
                schedule_date='2026-01-08',
                assigned_driver=driver).sort_values(['schedule_date', 'actual_start'])
df[['labor_context_id', 'schedule_date', 'actual_start', 'actual_end', 'start_point', 'end_point']]

,labor_context_id,schedule_date,actual_start,actual_end,start_point,end_point
1338,349836_free,2026-01-08 10:30:00-05:00,2026-01-08 09:00:00-05:00,2026-01-08 09:42:38.635063-05:00,POINT (-74.0286017 4.9203296),POINT (-74.0286017 4.9203296)
1339,349836_move,2026-01-08 10:30:00-05:00,2026-01-08 09:42:38.635063-05:00,2026-01-08 10:00:00-05:00,POINT (-74.0286017 4.9203296),POINT (-74.0651125 4.8512831)
1340,349836_labor,2026-01-08 10:30:00-05:00,2026-01-08 10:00:00-05:00,2026-01-08 10:51:27.004788-05:00,POINT (-74.0651125 4.8512831),POINT (-74.03727236805345 4.878226952715764)


In [32]:
labors_driver_df, moves_driver_df = filter_dfs_for_insertion(
    labors_algo_static_df, 
    moves_algo_static_df,
    city=city,
    fecha=date,
    created_at=new_service['created_at'],
    driver=driver
)

order = ['labor_id','labor_context_id','schedule_date', 'actual_start', 'actual_end', 'start_point', 'end_point']
order += [j for j in moves_driver_df.columns.tolist() if j not in order]
display(labors_driver_df)
display(moves_driver_df[order])

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,address_point,address_name,map_start_point,map_end_point,assigned_driver,actual_start,actual_end,dist_km,date,n_drivers
0,253756,349836,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,80663.0,2025-06-11 15:34:06.307000-05:00,2026-01-08 09:11:00-05:00,2026-01-08 11:45:00-05:00,11988,...,POINT (-74.0286017 4.9203296),casa,POINT (-74.0651125 4.8512831),POINT (-74.03727236805345 4.878226952715764),11988,2026-01-08 10:00:00-05:00,2026-01-08 10:51:27.004788-05:00,4.300053,2026-01-08,19


,labor_id,labor_context_id,schedule_date,actual_start,actual_end,start_point,end_point,service_id,labor_name,labor_category,assigned_driver,distance_km,duration_min,city,date
0,349836,349836_free,2026-01-08 10:30:00-05:00,2026-01-08 09:00:00-05:00,2026-01-08 09:42:38.635063-05:00,POINT (-74.0286017 4.9203296),POINT (-74.0286017 4.9203296),253756,FREE_TIME,FREE_TIME,11988,0.000000,42.6,149,2026-01-08
1,349836,349836_move,2026-01-08 10:30:00-05:00,2026-01-08 09:42:38.635063-05:00,2026-01-08 10:00:00-05:00,POINT (-74.0286017 4.9203296),POINT (-74.0651125 4.8512831),253756,DRIVER_MOVE,DRIVER_MOVE,11988,8.678041,17.4,149,2026-01-08
2,349836,349836_labor,2026-01-08 10:30:00-05:00,2026-01-08 10:00:00-05:00,2026-01-08 10:51:27.004788-05:00,POINT (-74.0651125 4.8512831),POINT (-74.03727236805345 4.878226952715764),253756,Alfred Initial Transport,VEHICLE_TRANSPORTATION,11988,NaN,51.5,149,2026-01-08


In [17]:
labor_iter = 0

# first_labor = labors_driver_df.iloc[labor_iter,:]

available_from = moves_driver_df['end_point'][labor_iter]
available_at = moves_driver_df['actual_end'][labor_iter]

available_from, available_at

('POINT (-74.0286017 4.9203296)',
 Timestamp('2026-01-08 09:42:38.635063-0500', tz='America/Bogota'))

In [13]:
next_service_start_pos = moves_driver_df['start_point'][labor_iter+3]
next_service_start_time = moves_driver_df['schedule_date'][labor_iter+3]

next_service_start_pos, next_service_start_time

('POINT (-74.0434496 4.703822299999999)',
 Timestamp('2026-01-08 09:00:00-0500', tz='America/Bogota'))

In [14]:
next_service_start_time <= new_service['schedule_date']

True

### Since the starting time of the next service is earlier than the schedule time of the new service, it is not 

In [15]:
labor_iter += 3

In [16]:
# first_labor = labors_driver_df.iloc[labor_iter,:]

available_from = moves_driver_df['end_point'][labor_iter]
available_at = moves_driver_df['actual_end'][labor_iter]

available_from, available_at

('POINT (-74.028923 4.704466)',
 Timestamp('2026-01-08 09:17:25.029013-0500', tz='America/Bogota'))

In [17]:
next_service_start_pos = moves_driver_df['start_point'][labor_iter+3]
next_service_start_time = moves_driver_df['schedule_date'][labor_iter+3]

next_service_start_pos, next_service_start_time

('POINT (-74.0343054 4.692379399999999)',
 Timestamp('2026-01-08 10:00:00-0500', tz='America/Bogota'))

In [18]:
next_service_start_time <= new_service['schedule_date']

True

### Since the starting time of the next service is earlier than the schedule time of the new service, it is not 

In [19]:
labor_iter += 3

In [20]:
# first_labor = labors_driver_df.iloc[labor_iter,:]

available_from = moves_driver_df['end_point'][labor_iter]
available_at = moves_driver_df['actual_end'][labor_iter]

available_from, available_at

('POINT (-74.06596379999999 4.6401052)',
 Timestamp('2026-01-08 10:25:11.051557-0500', tz='America/Bogota'))

In [21]:
next_service_start_pos = moves_driver_df['start_point'][labor_iter+3]
next_service_start_time = moves_driver_df['schedule_date'][labor_iter+3]

next_service_start_pos, next_service_start_time

('POINT (-74.10500789999999 4.6304576)',
 Timestamp('2026-01-08 14:00:00-0500', tz='America/Bogota'))

In [22]:
next_service_start_time <= new_service['schedule_date']

False

### This is the breaking point... this means that the next labor of the driver is after the scheduled time of the new labor. This is the point where we could potentially assign the new labor. 

### Now it's needed to check if the driver could make it from it's current position to the new service

In [23]:
distance_to_new_service, _ = distance(available_from, 
                                   new_service['start_address_point'],
                                   method='haversine')
travel_time = distance_to_new_service/ALFRED_SPEED*60

would_arrive_at = available_at + timedelta(minutes=travel_time)

In [25]:
would_arrive_at <= new_service['latest_arrival_time']

True

### The driver would be able to arrive to the service in time. Time to simulate the assignment and make sure he would arrive on time to the next service

In [27]:
new_service_travel_distance, _ = distance(new_service['start_address_point'],
                                       new_service['end_address_point'], 'haversine')

new_service_travel_time = new_service_travel_distance/VEHICLE_TRANSPORT_SPEED*60

total_new_service_duration = TIEMPO_ALISTAR + new_service_travel_time + TIEMPO_FINALIZACION

finish_new_service_time = would_arrive_at + timedelta(minutes=total_new_service_duration)
finish_new_service_pos = new_service['end_address_point']

In [28]:
finish_new_service_time

Timestamp('2026-01-08 11:23:01.646585-0500', tz='America/Bogota')

#### Compute driving time to the next scheduled_service

In [33]:
travel_distance_to_next_service, _ = distance(finish_new_service_pos, 
                                              next_service_start_pos,
                                              'haversine')
travel_time_to_next_service = travel_distance_to_next_service/ALFRED_SPEED*60
would_arrive_to_next_service_at = finish_new_service_time + timedelta(minutes=travel_distance_to_next_service)

In [38]:
would_arrive_to_next_service_at <= next_service_start_time + timedelta(minutes=TIEMPO_GRACIA)

True

In [ ]:
### LOGICA COMPLETA

